# Production Planning

In [1]:
from io import StringIO

import numpy as np
import pandas as pd
import pyomo.environ as pyo

# Data from Problem 12.3 from the book Model Building in Mathematical Programming (H. Paul Williams)
# Two entries in the top row changed

data = """
10 6 8 4 11 9 3
0.5 0.7 – – 0.3 0.2 0.5
0.1 0.2 – 0.3 – 0.6 –
0.2 – 0.8 – – – 0.6
0.05 0.03 – 0.07 0.1 – 0.08
 – – 0.01 – 0.05 – 0.05
 """

# Column names for the big Data Frame
columns = ["Profit", "Grinding", "V_drilling", "H_drilling", "Boring", "Planing"]

production = pd.read_csv(StringIO(data), header=None, sep="\s+", na_values=["–"])
production.fillna(0, inplace=True)
production = production.transpose()
production.columns = columns
production.reindex(list(range(production.shape[0])))
production.index.name = "Product"

profit = production["Profit"]
# A little modification to the profit data to make the products more "competitive" against each other
profit[0] = 7
profit[4] = 8

# Remove the Profit column from the data frame because we have it elsewhere
production.drop("Profit", axis=1, inplace=True)

# Numbers of machines of different types
nb_machines = pd.Series(
    [4, 2, 3, 1, 1], index=["Grinding", "V_drilling", "H_drilling", "Boring", "Planing"]
)
# total hours per month = 24 working days times 8 hours
hours = 24 * 8

display(production)
display(profit)

,Grinding,V_drilling,H_drilling,Boring,Planing
Product,,,,,
0,0.5,0.1,0.2,0.05,0.00
1,0.7,0.2,0.0,0.03,0.00
2,0.0,0.0,0.8,0.00,0.01
3,0.0,0.3,0.0,0.07,0.00
4,0.3,0.0,0.0,0.10,0.05
5,0.2,0.6,0.0,0.00,0.00
6,0.5,0.0,0.6,0.08,0.05


Product
0    7.0
1    6.0
2    8.0
3    4.0
4    8.0
5    9.0
6    3.0
Name: Profit, dtype: float64

## Nominal Model

In [4]:
# Solving the nominal production problem

m = pyo.ConcreteModel("Production planning")

products = list(profit.index)
resources = list(production.columns)

# Variables = how much of each product we make
m.p = pyo.Var(products, within=pyo.NonNegativeReals)

@m.Constraint(resources)
def machine_availability(m, resource):
    return pyo.quicksum((m.p[product] * production.loc[product, resource] for product in products), linear=True) \
        <= hours * nb_machines[resource]
    
@m.Objective(sense=pyo.maximize)
def total_profit(m):
    return pyo.quicksum((m.p[product] * profit.loc[product] for product in products), linear=True)

solver = pyo.SolverFactory("cbc")
solver.solve(m)

nominal_plan = pd.Series({i: m.p[i]() for i in products}, name="Nominal")
display(nominal_plan)

0       0.00000
1     117.79141
2     720.00000
3       0.00000
4    1884.66260
5     600.73620
6       0.00000
Name: Nominal, dtype: float64

## Robust Optimization

In [10]:
# We shall now solve the robust problem with each (product, machine) time deviating
# by at most max_perturbation * 100%,
# and per machine the  at most unc_budget products in total deviate by their max

max_perturbation = 0.05
unc_budget = 1

production_perturbation = production.applymap(lambda x: x * max_perturbation)

m = pyo.ConcreteModel("Production planning")

products = list(profit.index)
resources = list(production.columns)

# Old variables
m.p = pyo.Var(products, domain=pyo.NonNegativeReals)

# Variable which will act as a proxy on s >= abs(duration - nominal duration)
m.s = pyo.Var(products, resources, domain=pyo.NonNegativeReals)

# Dual variable related to the budget constraint in the uncertainty set
m.lam = pyo.Var(resources, domain=pyo.NonNegativeReals)

@m.Constraint(resources)
def machine_availability(m, resource):
    return unc_budget * m.lam[resource] \
            + pyo.quicksum((m.p[product] * production.loc[product, resource] for product in products), linear=True) \
            + pyo.quicksum((m.s[product, resource] for product in products), linear=True) \
            <= hours * nb_machines[resource]

@m.Constraint(products, resources)
def constraint2(m, product, resource):
    return m.s[product, resource] >= m.p[product] * production_perturbation.loc[product, resource] - m.lam[resource]

@m.Objective(sense=pyo.maximize)
def total_profit(m):
    return pyo.quicksum((m.p[product] * profit.loc[product] for product in products), linear=True)

solver = pyo.SolverFactory("cbc")
solver.solve(m)

# Extract the solution
robust_plan = pd.Series({i: m.p[i]() for i in products}, name="Robust")
production_plans = pd.concat([nominal_plan, robust_plan], axis=1)
production_plans.index.name = "Product"
display(production_plans)

,Nominal,Robust
Product,,
0,0.00000,0.00000
1,117.79141,128.26024
2,720.00000,685.71429
3,0.00000,0.00000
4,1884.66260,1791.92560
5,600.73620,568.80627
6,0.00000,0.00000


In [9]:
production_plans_profits = production_plans.apply(
    lambda x: sum([x[i] * profit[i] for i in x.index]), axis=0
)
display(production_plans_profits)

Nominal    26950.67506
Robust     25709.93699
dtype: float64